# Events dataset

Investigation over multiple possible sources:
- GDELT https://www.gdeltproject.org (not adapted)
- ACLED https://acleddata.com/
- Wikipedia (confirmed)

In [1]:
# Notebook config
%config Completer.use_jedi = False

In [8]:
import sys   
import os
from datetime import datetime
import re

import asyncio
import aiohttp 
import aiofiles 

import pandas as pd
import requests
import tqdm.asyncio
from bs4 import BeautifulSoup

In [9]:
# Configuration
DATA_PATH = "data"
PKL_PATH = os.path.join(DATA_PATH, "pkl")
CSV_PATH = os.path.join(DATA_PATH, "csv")
RESOURCES_PATH = os.path.join(DATA_PATH, "resources")
EVENTS_PATH = os.path.join(DATA_PATH, "events")

## 1. GDELT 
https://www.gdeltproject.org/data.html#rawdatafiles

Need to scrape the page to get the links and download each file.
Use async - await to make it a lot faster.

In [2]:
def parse_date(date_str):
    """
    Parse date from string
    """
    try:
        date_str = date_str.strip(".export.CSV.zip")
        date = datetime.strptime(date_str, "%Y%m%d")
    except ValueError:
        return
    return date

def in_time_range(date):
    """
    Return true if date is in time range
    """ 
    return bool(start_time <= date <= end_time)

def gen_link(a):
    """
    Get link from a html tag
    """
    base = "http://data.gdeltproject.org/events/"
    date = parse_date(a.text)
    
    if not date:
        return
    
    if in_time_range(date):
        return base + a.text
    return 
 
def get_urls():
    """
    Retrieve file urls from GDELT's website
    """
    url = "http://data.gdeltproject.org/events/index.html"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # Get the <a> html elements
    ax = soup.find_all("a")
    # Remove 3 first elems
    for _ in range(3):
        ax.pop(0)
    
    # Generate URLs from html tag
    urls = []
    for a in ax:
        new_a = gen_link(a)
        if new_a:
            urls.append(new_a)
    return urls

In [3]:
async def fetch_file(url, dirpath):
    """
    Download files asynchronously
    """
    fname = url.split("/")[-1]
    fpath = os.path.join(dirpath, fname)
    # Don't redownload if file already exists
    if os.path.exists(fpath):
        return
    
    async with sema, aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            assert resp.status == 200
            data = await resp.read()

    async with aiofiles.open(
        fpath, "wb"
    ) as outfile:
        await outfile.write(data)

In [ ]:
# Check that directory exists
if not os.path.exists(EVENTS_PATH):
    os.mkdir("./data/events")

# Update timerange of interest
# %Y, %m, %d
start_time = datetime(2015, 1, 1)
end_time = datetime(2020, 12, 31) 

# Get file urls
urls = get_urls()
# Sanity check
print(f"len: {len(urls)} \nfirst link: {urls[0]} \nlast link: {urls[-1]}")

# Download the files
sema = asyncio.BoundedSemaphore(5)
loop = asyncio.get_event_loop()
tasks = [loop.create_task(fetch_file(url, EVENTS_PATH)) for url in urls]

for fetch in tqdm.asyncio.tqdm.as_completed(tasks):
    await fetch

print("Done")

The data is actually not what I expected, it would take too much time to make it usable.

## 2. ACLED
https://acleddata.com/

Could export file specific to US using their website:
`2015-01-01-2020-12-31-North_America-United_States.csv`

In [27]:
df = pd.read_csv(os.path.join(EVENTS_PATH, "2015-01-01-2020-12-31-North_America-United_States.csv"))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26238 entries, 0 to 26237
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   data_id           26238 non-null  int64  
 1   iso               26238 non-null  int64  
 2   event_id_cnty     26238 non-null  object 
 3   event_id_no_cnty  26238 non-null  int64  
 4   event_date        26238 non-null  object 
 5   year              26238 non-null  int64  
 6   time_precision    26238 non-null  int64  
 7   event_type        26238 non-null  object 
 8   sub_event_type    26238 non-null  object 
 9   actor1            26238 non-null  object 
 10  assoc_actor_1     18110 non-null  object 
 11  inter1            26238 non-null  int64  
 12  interaction       26238 non-null  int64  
 13  region            26238 non-null  object 
 14  country           26238 non-null  object 
 15  admin1            26238 non-null  object 
 16  admin2            26192 non-null  object

Need to find a way to filter out those events and only retain the biggest ones.

In [41]:
df["year"].unique()

array([2020])

Even though not mentioned directly, they only provide data since 2020 for the US. So unusable too.

## 3. Wikipedia dataset
https://en.wikipedia.org/wiki/2020_in_United_States_politics_and_government


In [93]:
l_2015 = [ 
    ("2015 March 25","Indiana Governor Mike Pence signs Indiana SB 101 into law, prompting a national debate over religious freedom and LGBT rights. After a week of public outcry, the law is modified and similar bills in Arkansas, North Carolina, Georgia, and North Dakota fail to pass their state legislatures.", "LGBT debate"), 
    ("2015 April 11", " President Barack Obama and Cuban President Raúl Castro discuss historic proposed normalizations to Cuba-United States relations at the 7th Summit of the Americas in Panama City, Panama.", "US-Cuba relations start again"), 
    ("2015 April", "start of the 2016 presidential campaign", "2016 presidential campaign"), 
    ("2015 April 25", " Protests in Baltimore, Maryland from Baltimore City Hall to the Inner Harbor against the April 19 death of Freddie Gray in police custody soon turn violent, with extensive property damage, looting, numerous arrests, and injuries of both police officers and citizens being reported. Maryland Governor Larry Hogan declares a state of emergency in the city and deploys the Maryland National Guard in response.", "Baltimore protests"), 
    ("2015 May 22"," The State Department begins publishing the e-mails of Hillary Clinton, mostly concerning the 2012 Benghazi attack, that drew controversy earlier in the year when it was revealed that Clinton had been using a private e-mail server for work-related communications during her time as Secretary of State.", "H. Clinton email's controversy"), 
    ("2015 June 26"," In a 5, 4 decision, the U.S. Supreme Court rules that the Constitution guarantees a right to same-sex marriage in the United States. Justice Anthony Kennedy wrote the majority opinion, joined by Justices Ruth Bader Ginsburg, Stephen Breyer, Sonia Sotomayor and Elena Kagan.", "Same-sex marriage supreme court rule"), 
    ("2015 October 1","26-year-old Christopher Harper-Mercer opens fire at Umpqua Community College in Roseburg, Oregon. killing 9 people and injuring 9 others. It is the largest shooting at a college since the Virginia Tech massacre in 2007.", "Umpqua Community College shooting"), 
    ("2015 October 29", " Representative Paul Ryan takes over for John Boehner as Speaker of the House of Representatives after being elected following Boehner's resignation.", "Paul Ryan (rep.) elected Speaker of the House of Representatives"), 
    ("2015 December 2","2015 San Bernardino attack: 14 people are killed in a terrorist attack at a facility for the mentally disabled in San Bernardino, California. It is the deadliest mass shooting in the United States in 2015, and the deadliest since 2012.", "San Bernardino shooting"),
]
l_2016 = [
    ("2016 February 9 "," The New Hampshire primaries are held. Donald Trump wins the Republican primary, and Bernie Sanders wins the Democratic primary.", "New Hampshire primaries won by D. Trump"),
    ("2016 March 1 "," Super Tuesday", "Super Tuesday"),
    ("2016 March 16 "," President Obama nominates Merrick Garland to replace Antonin Scalia on the Supreme Court.", "Merrick Garland Supreme Court nomination"),
    ("2016 May 4 "," John Kasich suspends his presidential campaign, leaving Donald Trump the presumptive Republican nominee.", "D. Trump is last rep. nominee"),
    ("2016 May 25 "," An audit by the State Department Inspector General finds that Hillary Clinton violated directives from the department in her use of a private email server for government business during her time as Secretary of State. The Inspector General says that Clinton did not request approval to use the private server, and that such a request would have been denied because of security risks.", "Investigation against H. Clinton over private email use"),
    ("2016 June 12 ","Orlando shooting", "Orlando shooting"),
    ("2016 June 22", "Members of the House Democratic Caucus, led by Representative John Lewis (D-Georgia) and Representative Katherine Clark (D-Massachusetts) declare their intention to remain on the floor of the House of Representatives until its Republican Speaker, Paul Ryan, allows votes on gun control legislation in the aftermath of the Orlando nightclub shooting. The sit-in is staged by about 60 legislators.", "Democrat Representatives protest in House of Representatives"),
    ("2016 July 8 "," Despite the conclusions of the FBI and attorney general, the State Department reopens its investigation into Hillary Clinton's use of a private email server.", "FBI reopens investigation on H. Clinton"),
    ("2016 September 21 "," Charlotte protests", "Charlotte protests"),
    ("2016 September 26 "," Presidential candidates Donald Trump and Hillary Clinton take part in their first live televised debate, with an estimated audience of up to 100 million viewers.", "Presidential debate betwenn D. Trump and H. Clinton"),
    ("2016 November 8 "," 2016 presidential Election", "2016 Presidential Election"),
    ("2016 December 2 ","Donald Trump becomes the first US president or president-elect since 1979 to make direct contact with the President of Taiwan, upsetting Chinese diplomats and sparking concern over whether Trump will uphold the \"One China policy\" at the foundation of China-U.S. relations.", "D. Trump visits Taiwan"),
    ("2016 December 31 "," United States troops withdraw from Afghanistan, leaving behind 8,400 troops stationed at 4 garrisons (Kabul, Kandahar, Bagram, and Jalalabad).", "US troosp withdraw from Afghanistan"),
]
l_2017 = [
    ("2017 January 6"," After briefing President Barack Obama, the United States Senate, and President-elect Donald Trump, the United States Intelligence Community releases a declassified version of its investigation into Russia's interference in the 2016 presidential election. The report asserts that Russia carried out a massive cyber operation on orders from President Vladimir Putin to influence the election in favor of Trump in a multipronged attack consisting of hacking the Democratic National Committee, use of social media and Internet trolls to spread misinformation, and open propaganda on Russian state media outlets. Trump asserts that the outcome of the election was not affected by the interference, but nonetheless announces his intention to appoint a team to combat international cyber attacks within his first 90 days in office.", "Allegations of russian interference in presidential election"),
    ("2017 January 20"," Donald Trump is sworn in as the 45th President of the United States, and Mike Pence is sworn in as Vice President of the United States.", "D. Trump sworn as President"),
    ("2017 May "," FBI chief James Comey is fired by Trump for mishandling of the Hillary Clinton email controversy. However, critics accuse Trump of attempting to subvert the ongoing FBI investigation into possible ties between Russia and the Trump campaign.", "FBI chief James Comey fired"),
    ("2017 June 17","Charleston church shooting", "Charleston church shooting"),
    ("2017 July 18","The US Senate votes to start debating a new Republican healthcare bill to replace Obamacare.", "Senate votes on new Republican healthcare bill"),
    ("2017 September 5"," The Trump administration announces that the Deferred Action for Childhood Arrivals (DACA) immigration policy, which was set by the Obama administration in 2012, will be scrapped.", "New immigration policy bill by Trump administration"),
    ("2017 October 1"," Las Vegas mass shooting", "Las Vegas mass shooting"),
    ("2017 December 2"," San Bernardino shooting", "San Bernardino shooting"),
    ("2017 December 6"," In a speech at the White House, President Trump announces that the US now recognizes Jerusalem as Israel's capital.", "US recognire Jerusalem as Israel's capital"),
]
l_2018 = [
    ("2018 January 11","\"Shitholegate\" During a meeting with lawmakers about immigration, President Trump is reported to have asked, \"Why are we having all these people from shithole countries come here?\" His remarks are condemned as \"racist\" and \"shocking\" by a UN spokesman. (comp. Donald Trump racial views) The government of Botswana demands a clarification, and Ambassador Earl R. Miller is asked if the USDS regards Botswana as a \"shithole\" country.", "Shitholegate"),
    ("2018 January 20","Senate Democrats block a bill that would have kept the government running until mid-February and the government shutdown of January 2018 begins. ", "Government shutdown"),
    ("2018 February 14"," Marjory Stoneman Douglas High School in Parkland, Florida mass shooting", "Marjory Stoneman Douglas High School shooting"),
    ("2018 March 22","President Trump announces tariffs on up to $60bn in Chinese goods and plans to limit the country's investment in the US. The Dow Jones falls sharply in response.", "Tariffs against China"),
    ("2018 June 18","70 former U.S. Attorneys deliver a letter to Jeff Sessions, urging that he end his \"dangerous, expensive, zero tolerance\" migrant policy, which separates children at the border, calling it \"inconsistent with the values of the institution in which we served.\"", "Opposition to new migration policy"),
    ("2018 June 19","The United States announces that it will withdraw from the United Nations Human Rights Council.", "Announcement of US withdrawal from UNHCR"),
    ("2018 August 1","President Trump calls for the Russia investigation to end \"right now\", urging Attorney General Jeff Sessions to halt the inquiry into alleged election meddling, while accusing special counsel Robert Mueller of being \"totally conflicted\".", "Trump opposed to russian interference investigation"),
    ("2018 October 2","The Washington Post journalist Jamal Khashoggi is murdered inside the Saudi consulate in Istanbul, triggering a diplomatic crisis between the U.S. and Saudi Arabia.", "Murder of Jamal Kashoggi"),
    ("2018 October 20","President Trump announces that the US will \"terminate\" the Intermediate-Range Nuclear Forces Treaty over alleged Russian violations.", "US withdrawal of IRNFT treaty"),
    ("2018 October 29","Migrant caravan. 800 U.S. soldiers are deployed to the Mexico–United States border as a part of Operation Faithful Patriot, reinforcing the border against incoming Central American migrant caravans.", "US troops deployed to stop migrant caravans"),
    ("2018 November 6","Midterm elections", "Midterm elections"),
    ("2018 December 13","In a rare rebuke to the White House, the Senate votes 56–41 to end US military assistance to Saudi Arabia's intervention in Yemen over alleged war crimes. It passes a separate resolution that holds Saudi Crown Prince Mohammed bin Salman personally responsible for the death of Saudi dissident Jamal Kashoggi.", "US stops military assistance to Saudi Arabia"),
]
l_2019 = [
    ("2019 January 3", "The Democrats take control of the House of Representatives, with a promise to end the government shutdown, but without funding for President Donald Trump's proposed border wall. The Republicans increase their control of the United States Senate.", "Democrats in majority in the House of Representatives"),
    ("2019 January", "start of the 2020 presidential campaign", "2020 presidential campaign starts"),
    ("2019 January 28", "The Justice Department charges Chinese tech firm Huawei with multiple counts of fraud, raising U.S.China tensions.", "Huawei accused of fraud"),
    ("2019 March 22", "Special Counsel Robert Mueller turns in the report of his nearly two-year investigation on whether the Trump campaign helped Russia interfere in the 2016 presidential election.", "Mueller investigation given to Congress"),
    ("2019 April 18", "A redacted version of the final Mueller Report is released by Attorney General William Barr to Congress and the general public. The report goes into detail on findings from the 2017-2019 Special Counsel probe regarding Russian contacts with the 2016 Donald Trump election campaign and matters regarding obstruction of justice by President Trump himself.", "Redacted version of Mueller Report made available to the public"),
    ("2019 July 6", "Billionaire financier and registered sex offender Jeffrey Epstein is arrested on federal charges of sex trafficking and additionally charged by the Southern District of New York with sex trafficking and with conspiracy to traffic minors in Florida and New York.", "Jeffrey Epstein arrested"),
    ("2019 August 3", "El Paso shooting: A mass shooting occurs at a Walmart near the Cielo Vista Mall in El Paso, Texas, resulting in 23 fatalities and 23 injured.", "El Paso shooting"),
    ("2019 August 12", "The Trump administration issues new rules that reject applicants for temporary or permanent visas for failing to meet income standards or for receiving public assistance such as welfare, food stamps, public housing or Medicaid.", "New rules for visa applications"),
    ("2019 September 24", "Speaker of the House Nancy Pelosi announces the start of a formal impeachment inquiry against President Trump.", "Start of impeachment inquiry against Pres. D. Trump"),
    ("2019 October 3", "President Trump calls on Ukraine and China to investigate former Vice President Joe Biden and his son Hunter Biden.", "Trump asks other countries to investigate J. Biden"),
    ("2019 December 1", "The White House announces that President Donald Trump will not participate in the House Judiciary Committee impeachment hearing on December 3.", "D. Trump refuses to appear for to his impeachment hearing"),
    ("2019 December 3", "The House Intelligence Committee releases a report accusing President Donald Trump of using his office to further his personal interests.", "Allegations of abuse of power from D. Trump"),
    ("2019 December 6", "The House of Representatives approves 228-187 to restore parts of the Voting Rights Act of 1965. President Trump threatens to veto if the bill is approved by the Senate.", "House of Representatives opposes Trump administration"),
    ("2019 December 10", "Democrats in the House of Representatives announce formal charges against President Trump that accuse him of abusing power and obstructing Congress, making him the fourth U.S. president in history to face impeachment.", "Democrats accuse Trump of abusing power"),
    ("2019 December 18", "The U.S. House of Representatives votes largely along party lines to forward two articles of impeachment against President Donald Trump to the Senate, accusing him of abuse of power and obstructing Congress. Donald Trump becomes the third U.S. president to be impeached by the House.", "Democrats continue impeachment procedure"),
]
l_2020 = [
    ("2020 January 3 ","2019–2021 Persian Gulf crisis: President Donald Trump approves the targeted killing of Iranian general Qasem Soleimani and Iraqi paramilitary leader Abu Mahdi al-Muhandis in Baghdad, Iraq.", "Murder of Iranian general Qasem Soleimani"),
    ("2020 January 9 ","Persian Gulf crisis: The House of Representatives votes 224–194 to pass a non-binding War Powers Resolution to limit the president's ability to pursue military actions against Iran without congressional consent.", "House of Representatives limits Pres. power to start war"),
    ("2020 January 14 ","The seventh Democratic presidential debate is held in Des Moines, Iowa. Six candidates participate.", "Seventh Democratic debate"),
    ("2020 January 16 ","The impeachment trial of President Donald Trump begins in the U.S. Senate.", "First impeachment trial begins"),
    ("2020 January 21 ","COVID-19 pandemic: The first case of COVID-19 in the United States is confirmed by the Centers for Disease Control and Prevention (CDC).", "First case of COVID-19 discovered in the US"),
    ("2020 february 5 ","The impeachment trial of President Donald Trump concludes with the Senate voting 52–48 to acquit on the first article of impeachment and 53–47 on the second charge. Utah Senator Mitt Romney becomes the first ever senator to vote to remove a president of their own political party.", "Trumps acquitted"),
    ("2020 April 22 ","COVID-19 pandemic: President Trump signs an immigration executive order halting the issuance of certain green cards for 60 days.", "Harder immigration policy"),
    ("2020 May 25 ","Minneapolis police officer Derek Chauvin murders George Floyd, 46, by kneeling on his neck, as three other officers watch. Video of the incident goes viral and the four officers are subsequently fired.", "Start of George Floyd protests"),
    ("2020 September 26 ","President Trump nominates Seventh Circuit Court of Appeals Judge Amy Coney Barrett to fill the vacant Supreme Court seat following the death of Judge Ruth Bader Ginsburg.", "Trump nominates new Federal judge"),
    ("2020 November 3 ","presidential election", "2020 president election"),
    ("2020 November 4 ","The United States formally withdraws from the Paris Agreement.", "US officially withdraw from Paris Agreement"),
    ("2020 November 7 ","Joe biden won the election", "Joe Biden victory"),
]

In [94]:
events_lst = []
for lst in [l_2015, l_2016, l_2017, l_2018, l_2019, l_2020]:
    [events_lst.append(event) for event in lst]
print(len(events_lst), "events in list")

70 events in list


In [97]:
events = pd.DataFrame(events_lst, columns=["date", "description", "short_desc"])
events["date"] = pd.to_datetime(events["date"]) 
display(events.info())
display(events.sample(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         70 non-null     datetime64[ns]
 1   description  70 non-null     object        
 2   short_desc   70 non-null     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 1.8+ KB


None

,date,description,short_desc
36,2018-06-19,The United States announces that it will withd...,Announcement of US withdrawal from UNHCR
44,2019-01-01,start of the 2020 presidential campaign,2020 presidential campaign starts
38,2018-10-02,The Washington Post journalist Jamal Khashoggi...,Murder of Jamal Kashoggi


In [99]:
# Save as csv
events.to_csv(os.path.join(EVENTS_PATH, "events.csv"))

[2015](https://en.wikipedia.org/wiki/2015_in_the_United_States):
- March 25 Indiana Governor Mike Pence signs Indiana SB 101 into law, prompting a national debate over religious freedom and LGBT rights. After a week of public outcry, the law is modified and similar bills in Arkansas, North Carolina, Georgia, and North Dakota fail to pass their state legislatures.[83][84][85]
- April 11 – President Barack Obama and Cuban President Raúl Castro discuss historic proposed normalizations to Cuba-United States relations at the 7th Summit of the Americas in Panama City, Panama.
- April - start of the 2016 presidential campaign
- April 25 – May 3 – Protests in Baltimore, Maryland from Baltimore City Hall to the Inner Harbor against the April 19 death of Freddie Gray in police custody soon turn violent, with extensive property damage, looting, numerous arrests, and injuries of both police officers and citizens being reported. Maryland Governor Larry Hogan declares a state of emergency in the city and deploys the Maryland National Guard in response.[99][100]
- May 22 - The State Department begins publishing the e-mails of Hillary Clinton, mostly concerning the 2012 Benghazi attack, that drew controversy earlier in the year when it was revealed that Clinton had been using a private e-mail server for work-related communications during her time as Secretary of State.[127]
- June 24 - The House of Representatives passes a bill that gives President Barack Obama an authority to negotiate international trade agreements with Congress's approval. It is seen as a stepping stone toward the passing of Obama's proposed trade deal with eleven Asian countries known as the Trans-Pacific Partnership.[169]
- June 26 - In a 5–4 decision, the U.S. Supreme Court rules that the Constitution guarantees a right to same-sex marriage in the United States. Justice Anthony Kennedy wrote the majority opinion, joined by Justices Ruth Bader Ginsburg, Stephen Breyer, Sonia Sotomayor and Elena Kagan.[173][174]
- October 1 - 26-year-old Christopher Harper-Mercer opens fire at Umpqua Community College in Roseburg, Oregon. killing 9 people and injuring 9 others. It is the largest shooting at a college since the Virginia Tech massacre in 2007.
- October 29 – Representative Paul Ryan takes over for John Boehner as Speaker of the House of Representatives after being elected following Boehner's resignation.[264]
- December 2 - 2015 San Bernardino attack: 14 people are killed in a terrorist attack at a facility for the mentally disabled in San Bernardino, California. It is the deadliest mass shooting in the United States in 2015, and the deadliest since 2012.

[2016](https://en.wikipedia.org/wiki/2016_in_the_United_States):
- February 9 – The New Hampshire primaries are held. Donald Trump wins the Republican primary, and Bernie Sanders wins the Democratic primary.
- March 1 – Super Tuesday
- March 16 – President Obama nominates Merrick Garland to replace Antonin Scalia on the Supreme Court.[34]
- May 4 - John Kasich suspends his presidential campaign, leaving Donald Trump the presumptive Republican nominee.[97]
- May 25 – An audit by the State Department Inspector General finds that Hillary Clinton violated directives from the department in her use of a private email server for government business during her time as Secretary of State. The Inspector General says that Clinton did not request approval to use the private server, and that such a request would have been denied because of security risks.[115]
- June 12 - Orlando shooting
- June 22–23 – Members of the House Democratic Caucus, led by Representative John Lewis (D-Georgia) and Representative Katherine Clark (D-Massachusetts), declare their intention to remain on the floor of the House of Representatives until its Republican Speaker, Paul Ryan, allows votes on gun control legislation in the aftermath of the Orlando nightclub shooting. The sit-in is staged by about 60 legislators.[127]
- July 8 – Despite the conclusions of the FBI and attorney general, the State Department reopens its investigation into Hillary Clinton's use of a private email server.[144]
- September 21 - Charlotte protests
- September 26 – Presidential candidates Donald Trump and Hillary Clinton take part in their first live televised debate, with an estimated audience of up to 100 million viewers.[173]
- November 8 - 2016 presidential Election
- December 2 - Donald Trump becomes the first US president or president-elect since 1979 to make direct contact with the President of Taiwan, upsetting Chinese diplomats and sparking concern over whether Trump will uphold the "One China policy" at the foundation of China-U.S. relations.[205]
- December 31 – United States troops withdraw from Afghanistan, leaving behind 8,400 troops stationed at 4 garrisons (Kabul, Kandahar, Bagram, and Jalalabad).

[2017](https://en.wikipedia.org/wiki/2017_in_the_United_States):
- January 6 - After briefing President Barack Obama, the United States Senate, and President-elect Donald Trump, the United States Intelligence Community releases a declassified version of its investigation into Russia's interference in the 2016 presidential election. The report asserts that Russia carried out a massive cyber operation on orders from President Vladimir Putin to influence the election in favor of Trump in a multipronged attack consisting of hacking the Democratic National Committee, use of social media and Internet trolls to spread misinformation, and open propaganda on Russian state media outlets. Trump asserts that the outcome of the election was not affected by the interference, but nonetheless announces his intention to appoint a team to combat international cyber attacks within his first 90 days in office.[6][7][8]
- January 20 – Donald Trump is sworn in as the 45th President of the United States,[27] and Mike Pence is sworn in as Vice President of the United States.
- May 9 - FBI chief James Comey is fired by Trump for mishandling of the Hillary Clinton email controversy. However, critics accuse Trump of attempting to subvert the ongoing FBI investigation into possible ties between Russia and the Trump campaign.[98]
- June 17 - Charleston church shooting
- July 18 – The US Senate votes to start debating a new Republican healthcare bill to replace Obamacare.[136]
- September 5 – The Trump administration announces that the Deferred Action for Childhood Arrivals (DACA) immigration policy, which was set by the Obama administration in 2012, will be scrapped.[206]
- October 1 - Las Vegas mass shooting
- December 2 - San Bernardino shooting
- December 6 – In a speech at the White House, President Trump announces that the US now recognizes Jerusalem as Israel's capital.[272]

[2018](https://en.wikipedia.org/wiki/2018_in_the_United_States):
- January 11 – "Shitholegate" During a meeting with lawmakers about immigration, President Trump is reported to have asked, "Why are we having all these people from shithole countries come here?"[9][10] His remarks are condemned as "racist" and "shocking" by a UN spokesman.[11] (comp. Donald Trump racial views) The government of Botswana demands a clarification,[12] and Ambassador Earl R. Miller is asked if the USDS regards Botswana as a "shithole" country.[13] The event is termed by many media outlets "Shitholegate".[14]
- January 20 – Senate Democrats block a bill that would have kept the government running until mid-February and the government shutdown of January 2018 begins. [21]
- February 14 -  Marjory Stoneman Douglas High School in Parkland, Florida mass shooting
- March 22 – President Trump announces tariffs on up to $60bn in Chinese goods and plans to limit the country's investment in the US. The Dow Jones falls sharply in response.[67]
- The Senate Intelligence Committee releases an unclassified version of its investigation into Russian cyberattacks in 2016, concluding: "Russian-affiliated cyber actors were able to gain access to restricted elements of election infrastructure. [...] In a small number of states, these cyber actors were in a position to, at a minimum, alter or delete voter registration data; however, they did not appear to be in a position to manipulate individual votes or aggregate vote totals."[109]
- June 18 - 70 former U.S. Attorneys deliver a letter to Jeff Sessions, urging that he end his "dangerous, expensive, zero tolerance" migrant policy, which separates children at the border, calling it "inconsistent with the values of the institution in which we served."[139]
- June 19 - The United States announces that it will withdraw from the United Nations Human Rights Council.[141][142]
- August 1 – President Trump calls for the Russia investigation to end "right now", urging Attorney General Jeff Sessions to halt the inquiry into alleged election meddling, while accusing special counsel Robert Mueller of being "totally conflicted".[171]
- October 2 – The Washington Post journalist Jamal Khashoggi is murdered inside the Saudi consulate in Istanbul, triggering a diplomatic crisis between the U.S. and Saudi Arabia.
- October 20 – President Trump announces that the US will "terminate" the Intermediate-Range Nuclear Forces Treaty over alleged Russian violations.[221]
- October 29 – Migrant caravan. 800 U.S. soldiers are deployed to the Mexico–United States border as a part of Operation Faithful Patriot, reinforcing the border against incoming Central American migrant caravans.[228]
- November 6 - Midterm elections
- December 13 – In a rare rebuke to the White House, the Senate votes 56–41 to end US military assistance to Saudi Arabia's intervention in Yemen over alleged war crimes. It passes a separate resolution that holds Saudi Crown Prince Mohammed bin Salman personally responsible for the death of Saudi dissident Jamal Kashoggi.[251]

[2019](https://en.wikipedia.org/wiki/2019_in_the_United_States):
- January 3 – The Democrats take control of the House of Representatives, with a promise to end the government shutdown, but without funding for President Donald Trump's proposed border wall.[4] The Republicans increase their control of the United States Senate.[5]
- January - start of the 2020 presidential campaign
- January 28 – The Justice Department charges Chinese tech firm Huawei with multiple counts of fraud, raising U.S.-China tensions.[19]
- March 22 - Special Counsel Robert Mueller turns in the report of his nearly two-year investigation on whether the Trump campaign helped Russia interfere in the 2016 presidential election.[67]
- April 18 – A redacted version of the final Mueller Report is released by Attorney General William Barr to Congress and the general public. The report goes into detail on findings from the 2017–2019 Special Counsel probe regarding Russian contacts with the 2016 Donald Trump election campaign and matters regarding obstruction of justice by President Trump himself.[105][106]
- During a three-way meeting at the Korean Demilitarized Zone with South and North Korean leaders, President Trump becomes the first sitting U.S. president to set foot on North Korean territory. Both Trump and North Korean leader Kim Jong-un pledge to restart stalled nuclear negotiations between the two countries.[180]
- July 6 - Billionaire financier and registered sex offender Jeffrey Epstein is arrested on federal charges of sex trafficking and additionally charged by the Southern District of New York with sex trafficking and with conspiracy to traffic minors in Florida and New York.[187][188]
- August 3 - El Paso shooting: A mass shooting occurs at a Walmart near the Cielo Vista Mall in El Paso, Texas, resulting in 23 fatalities and 23 injured.[227][228]
- August 10 - 2020 Democratic candidates for president speak at a gun forum sponsored by Everytown for Gun Safety, in Des Moines, Iowa.[235]
- August 12 - The Trump administration issues new rules that reject applicants for temporary or permanent visas for failing to meet income standards or for receiving public assistance such as welfare, food stamps, public housing or Medicaid.[238]
- September 24 - Speaker of the House Nancy Pelosi announces the start of a formal impeachment inquiry against President Trump.[307]
- September 25 – The White House releases details of a July 25 phone call between President Trump and Volodymyr Zelensky, in which Trump asks the Ukrainian President to investigate Democratic presidential candidate Joe Biden.[309]
- October 3 - President Trump calls on Ukraine and China to investigate former Vice President Joe Biden and his son Hunter Biden.[332]
- December 1 - The White House announces that President Donald Trump will not participate in the House Judiciary Committee impeachment hearing on December 3.[441][442]
- December 3 - The House Intelligence Committee releases a report accusing President Donald Trump of using his office to further his personal interests.[450][451]
- Decdember 6 - The House of Representatives approves 228–187 to restore parts of the Voting Rights Act of 1965.[456] President Trump threatens to veto if the bill is approved by the Senate.[457]
- December 10 - Democrats in the House of Representatives announce formal charges against President Trump that accuse him of abusing power and obstructing Congress, making him the fourth U.S. president in history to face impeachment.[461]
- December 18 - The U.S. House of Representatives votes largely along party lines to forward two articles of impeachment against President Donald Trump to the Senate, accusing him of abuse of power and obstructing Congress. Donald Trump becomes the third U.S. president to be impeached by the House.[478]

[2020](https://en.wikipedia.org/wiki/2020_in_the_United_States):
- January 3 – 2019–2021 Persian Gulf crisis: President Donald Trump approves the targeted killing of Iranian general Qasem Soleimani and Iraqi paramilitary leader Abu Mahdi al-Muhandis in Baghdad, Iraq.[5]
- January 9 - Persian Gulf crisis: The House of Representatives votes 224–194 to pass a non-binding War Powers Resolution to limit the president's ability to pursue military actions against Iran without congressional consent.[11]
- January 14 - The seventh Democratic presidential debate is held in Des Moines, Iowa.[16] Six candidates participate.[17]
- January 16 – The impeachment trial of President Donald Trump begins in the U.S. Senate.[22]
- January 21 – COVID-19 pandemic: The first case of COVID-19 in the United States is confirmed by the Centers for Disease Control and Prevention (CDC).[24]
- february 5 - February 5 – The impeachment trial of President Donald Trump concludes with the Senate voting 52–48 to acquit on the first article of impeachment and 53–47 on the second charge. Utah Senator Mitt Romney becomes the first ever senator to vote to remove a president of their own political party.[38]
- April 22 – COVID-19 pandemic: President Trump signs an immigration executive order halting the issuance of certain green cards for 60 days.[161]
- May 25 – Minneapolis police officer Derek Chauvin murders George Floyd, 46, by kneeling on his neck, as three other officers watch. Video of the incident goes viral and the four officers are subsequently fired.[189]
- July 21 – Republican Ohio House Speaker Larry Householder is arrested by federal agents in connection with a $60 million bribery case. Former Ohio GOP Chairman Matt Borges is also arrested, along with a GOP advisor and two lobbyists.[259]
- September 26 – President Trump nominates Seventh Circuit Court of Appeals Judge Amy Coney Barrett to fill the vacant Supreme Court seat following the death of Judge Ruth Bader Ginsburg.[313]
- November 3 - presidential elections
- November 4 – The United States formally withdraws from the Paris Agreement.[371]
- November 7 - Joe biden won the election